In [68]:
import pandas as pd
import numpy as np
%matplotlib inline

In [69]:
# загрузка информации лога звонков
# columns = {0: "Calldate",1: "Source",2: "Destination",3:"Disconnect Time",4:"origCause_value",5:"destCause_value",6:"origDeviceName",7:"destDeviceName",8:"outpulsedCallingPartyNumber",9:"outpulsedCalledPartyNumber",10:"Duration"}
columns = ["Calldate","Source","Destination","Disconnect Time","origCause_value",
           "destCause_value","origDeviceName","destDeviceName","outpulsedCallingPartyNumber",
           "outpulsedCalledPartyNumber","Duration","No"]
dtypes = {"Calldate":"object","Source":"str","Destination":"object","Disconnect Time":"object","origCause_value":"object",
           "destCause_value":"object","origDeviceName":"object","destDeviceName":"object","outpulsedCallingPartyNumber":"object",
           "outpulsedCalledPartyNumber":"object","Duration":"int64","No":"object"}
log_zvonkov = pd.read_csv( 'report-2017-11-01-2017-11-16.csv', ';', header = None,names=columns, dtype=dtypes)
# log_zvonkov.head()
# log_zvonkov.dtypes
new_log = log_zvonkov[["Calldate","Source","Destination","Duration"]] # выбираем только нужные нам поля таблицы
new_log.head()
new_log.dtypes
# new_log.to_csv("logs.csv")

Calldate       object
Source         object
Destination    object
Duration        int64
dtype: object

In [70]:
# фильтрация по дате и времени

begin_date = "2017-11-02"
end_date = "2017-11-03"

begin_datetime = "{} 00:00:01".format(begin_date)
end_datetime = "{} 23:59:59".format(end_date)

filter_date = (new_log["Calldate"]>begin_datetime) & (new_log["Calldate"]<end_datetime)
new_log = new_log[filter_date]

# END фильтрация по дате

In [71]:
interval_time = (("13:00", "23:59"), ("13:00", "13:29"), ("13:30", "13:59"), ("14:00", "14:29"), ("14:30", "14:59"),
                     ("15:00", "15:29"), ("15:30", "15:59"), ("16:00", "23:59"))
name_sheets = ("лог звонков(итоговый)", "время 9-00 до 9-30", "время 9-30 до 10-00", "время 10-00 до 10-30",
                   "время 10-30 до 11-00",
                   "время 11-00 до 11-30", "время 11-30 до 12-00", "время 12-00 до 23-59")

In [72]:
# загрузка информации о принадлежности номеров телефонов к конкретным менеджерам
columns = ["Source","FioMPP","FioRg","Plan_result_unik_zvonok",""]
dtypes={"Source":"str","FioMPP":"object","FioRg":"object","Plan_result_unik_zvonok":"int64"}
list_cfg = pd.read_csv( 'list-num-tel.cfg', ';',header = None,names=columns, dtype = dtypes)
list_cfg = list_cfg[columns[:-1]] # отсекаем последний столбец
list_cfg.head()
# list_cfg.dtypes

,Source,FioMPP,FioRg,Plan_result_unik_zvonok
0,15106,Фахретдинов Данил,Бушуев Александр,5
1,15127,Моисеев Алексей,Бушуев Александр,5
2,15154,Шакиров Айнур,Бушуев Александр,5
3,15126,Джонбеков Джонбек,Дворников Александр,5
4,15103,Гасимова Лейсан,Дворников Александр,5


In [73]:
data = new_log.merge(list_cfg,on="Source",how="left")
data.head()
data = data.dropna() # удаление отсутствующих данных,таким образом отфильтровали номера которые нас не интересуют
data.to_csv("logs-fil.csv")
data.head()

,Calldate,Source,Destination,Duration,FioMPP,FioRg,Plan_result_unik_zvonok
8458,2017-11-02 12:30:25,15154,+79600413283,67,Шакиров Айнур,Бушуев Александр,5.0
11336,2017-11-02 12:59:01,15154,+74957820130,72,Шакиров Айнур,Бушуев Александр,5.0
11482,2017-11-02 13:00:26,17101,+79600904757,0,Стальновская Елена,Кочергин Владислав,5.0
11660,2017-11-02 13:02:37,17101,+78362636469,17,Стальновская Елена,Кочергин Владислав,5.0
11773,2017-11-02 13:03:45,15154,+79495777255,0,Шакиров Айнур,Бушуев Александр,5.0


In [74]:
# выборка общее кол-во набранных телефонов каждым из сотрудников
group = data["Destination"].groupby(data["Source"])  
group.count()
# END выборка общее кол-во набранных телефонов каждым из сотрудников



Source
15103     55
15105     48
15106     28
15112     45
15115     61
15119     76
15124     95
15126     47
15131     63
15132     72
15137     33
15141     99
15142     99
15143    109
15148     72
15149     69
15154     27
15156     59
15158     85
15164     60
15209     91
17101    156
17102     14
17104    103
17108     70
17412    108
17415     96
Name: Destination, dtype: int64

In [91]:
# подсчет уникальных звонков
result_unuque = group.nunique()
result_unuque

# END подсчет уникальных звонков

Source
15103    43
15105    37
15106    23
15112    32
15115    49
15119    59
15124    72
15126    28
15131    41
15132    58
15137    22
15141    75
15142    80
15143    84
15148    61
15149    51
15154    18
15156    46
15158    63
15164    41
15209    65
17101    96
17102    11
17104    62
17108    52
17412    82
17415    65
Name: Destination, dtype: int64

In [89]:
result_unuque_tel = result_unuque.to_frame()
# result_unuque_tel.rename(columns = {"Source","Unique_tel"})
result_unuque_tel.values


array([[43],
       [37],
       [23],
       [32],
       [49],
       [59],
       [72],
       [28],
       [41],
       [58],
       [22],
       [75],
       [80],
       [84],
       [61],
       [51],
       [18],
       [46],
       [63],
       [41],
       [65],
       [96],
       [11],
       [62],
       [52],
       [82],
       [65]])

In [ ]:
# не работает, сделать
result_table_log = list_cfg.merge(result_unuque_tel,on="Source",how="left")
result_table_log

In [53]:
%quickref